# Installing requirements

In [54]:
from tabulate import tabulate
import mysql.connector as mysql
from datetime import datetime
import os
import pandas as pd

In [19]:
#os.chdir("..")

In [20]:
#os.getcwd()

'/Users/victoriastasik/Documents/Very_large_distributed_data_volumes_exercise2'

In [5]:
#path = os.getcwd()
#os.chdir(path+"/code")

In [6]:
#os.getcwd()

In [7]:
#%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\sarac\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
#os.chdir("..")
#os.getcwd()

'/Users/victoriastasik/Documents/Very_large_distributed_data_volumes_exercise2'

# Class DBConnector

In [21]:
class DbConnector:
    """
    Connects to the MySQL server on the Ubuntu virtual machine.
    Connector needs HOST, DATABASE, USER and PASSWORD to connect,
    while PORT is optional and should be 3306.

    Example:
    HOST = "tdt4225-00.idi.ntnu.no" // Your server IP address/domain name
    DATABASE = "testdb" // Database name, if you just want to connect to MySQL server, leave it empty
    USER = "testuser" // This is the user you created and added privileges for
    PASSWORD = "test123" // The password you set for said user
    """

    def __init__(self,
                 HOST="tdt4225-19.idi.ntnu.no",
                 DATABASE="test_19",
                 USER="team19",
                 PASSWORD="team19*"):
        # Connect to the database
        try:
            self.db_connection = mysql.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD, port=3306)
        except Exception as e:
            print("ERROR: Failed to connect to db:", e)

        # Get the db cursor
        self.cursor = self.db_connection.cursor()

        print("Connected to:", self.db_connection.get_server_info())
        # get database information
        self.cursor.execute("select database();")
        database_name = self.cursor.fetchone()
        print("You are connected to the database:", database_name)
        print("-----------------------------------------------\n")

    def close_connection(self):
        # close the cursor
        self.cursor.close()
        # close the DB connection
        self.db_connection.close()
        print("\n-----------------------------------------------")
        print("Connection to %s is closed" % self.db_connection.get_server_info())


# Example program

In [22]:
class ExampleProgram:

    def __init__(self):
        self.connection = DbConnector()
        self.db_connection = self.connection.db_connection
        self.cursor = self.connection.cursor

    def create_table(self, table_name):
        query = """CREATE TABLE IF NOT EXISTS %s (
                   id INT AUTO_INCREMENT NOT NULL PRIMARY KEY,
                   name VARCHAR(30))
                """
        # This adds table_name to the %s variable and executes the query
        self.cursor.execute(query % table_name)
        self.db_connection.commit()

    def insert_data(self, table_name):
        names = ['Bobby', 'Mc', 'McSmack', 'Board']
        for name in names:
            # Take note that the name is wrapped in '' --> '%s' because it is a string,
            # while an int would be %s etc
            query = "INSERT INTO %s (name) VALUES ('%s')"
            self.cursor.execute(query % (table_name, name))
        self.db_connection.commit()

    def fetch_data(self, table_name):
        query = "SELECT * FROM %s"
        self.cursor.execute(query % table_name)
        rows = self.cursor.fetchall()
        print("Data from table %s, raw format:" % table_name)
        print(rows)
        # Using tabulate to show the table in a nice way
        print("Data from table %s, tabulated:" % table_name)
        print(tabulate(rows, headers=self.cursor.column_names))
        return rows

    def drop_table(self, table_name):
        print("Dropping table %s..." % table_name)
        query = "DROP TABLE %s"
        self.cursor.execute(query % table_name)

    def show_tables(self):
        self.cursor.execute("SHOW TABLES")
        rows = self.cursor.fetchall()
        print(tabulate(rows, headers=self.cursor.column_names))


def main():
    program = None
    try:
        program = ExampleProgram()
        program.create_table(table_name="Person")
        program.insert_data(table_name="Person")
        _ = program.fetch_data(table_name="Person")
        program.drop_table(table_name="Person")
        # Check that the table is dropped
        program.show_tables()
    except Exception as e:
        print("ERROR: Failed to use database:", e)
    finally:
        if program:
            program.connection.close_connection()


In [23]:
if __name__ == '__main__':
    main()

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('test_19',)
-----------------------------------------------

Data from table Person, raw format:
[(1, 'Bobby'), (2, 'Mc'), (3, 'McSmack'), (4, 'Board')]
Data from table Person, tabulated:
  id  name
----  -------
   1  Bobby
   2  Mc
   3  McSmack
   4  Board
Dropping table Person...
Tables_in_test_19
-------------------
Activity
Activity_l
TrackPoint
TrackPoint_l
User
User_l

-----------------------------------------------
Connection to None is closed


# Task 1

In [24]:
# we go to the Data folder
path = os.getcwd()
os.chdir(path+"/dataset/dataset/Data")
print(os.getcwd())

/Users/victoriastasik/Documents/Very_large_distributed_data_volumes_exercise2/dataset/dataset/Data


In [25]:
def get_info_from_labels_txt_file() :
    user_id_list = []
    transportation_mode_list = []
    start_date_time_list = []
    end_date_time_list = []

    for dirpath, dirnames, filenames in os.walk(os.getcwd()):
        for filename in filenames:
            # if there is a labels.txt file, we save the info
            if filename.endswith('.txt'):
                try :
                    with open(os.path.join(dirpath, filename)) as f:
                        lines = f.readlines()

                        # skip the header
                        lines = lines[1:]

                        for line in lines:
                            
                            # we get the info of one line
                            data = line.split()

                            # we save each information into the correct list
                            last_directory_name = os.path.basename(dirpath)
                            user_id_list.append(last_directory_name)

                            transportation_mode_list.append(data[4])

                            start_date = data[0]
                            start_time = data[1]
                            end_date = data[2]
                            end_time = data[3]
                            start_datetime_str = start_date + " " + start_time
                            end_datetime_str = end_date + " " + end_time
                            start_datetime_str = start_datetime_str.replace('/', '-')
                            end_datetime_str = end_datetime_str.replace('/', '-')
                            combined_start_datetime = datetime.strptime(start_datetime_str, "%Y-%m-%d %H:%M:%S")
                            combined_end_datetime = datetime.strptime(end_datetime_str, "%Y-%m-%d %H:%M:%S")

                            start_date_time_list.append(combined_start_datetime)
                            end_date_time_list.append(combined_end_datetime)
                    f.close()

                # error handling
                except FileNotFoundError:
                    print(f"the file {filename} doesn't exist.")
                except Exception as e:
                    print(f"Error: {e}")

    return user_id_list, transportation_mode_list, start_date_time_list, end_date_time_list

In [55]:
user_id_list, transportation_mode_list, start_date_time_list, end_date_time_list = get_info_from_labels_txt_file()

# each index of each list must correspond so the length of each list must be the same
print('len user: ' + str(len(user_id_list)))
print('len transportation mode: ' + str(len(transportation_mode_list)))
print('len start data time: ' + str(len(start_date_time_list)))
print('len end data time: ' + str(len(end_date_time_list)))

# we convert the lists into a dataframe
labels_txt = {'user_id': user_id_list, 'transportation_mode': transportation_mode_list, 'start_datetime': start_date_time_list, 'end_datetime': end_date_time_list}
labels_txt_df = pd.DataFrame(labels_txt)

len user: 14718
len transportation mode: 14718
len start data time: 14718
len end data time: 14718


In [56]:
labels_txt_df

,user_id,transportation_mode,start_datetime,end_datetime
0,104,bus,2007-11-30 09:59:00,2007-11-30 10:38:00
1,104,bus,2007-11-30 12:40:00,2007-11-30 13:18:00
2,104,bus,2007-12-02 12:27:00,2007-12-02 12:57:00
3,104,bus,2007-12-02 13:00:00,2007-12-02 13:25:00
4,104,car,2007-12-04 07:45:00,2007-12-04 08:30:00
...,...,...,...,...
14713,147,walk,2011-05-18 13:00:25,2011-05-18 13:07:45
14714,147,bus,2011-05-18 13:07:50,2011-05-18 13:21:55
14715,147,walk,2011-05-19 00:44:58,2011-05-19 01:00:43
14716,147,bus,2011-05-19 01:00:48,2011-05-19 01:17:48


In [82]:
def get_info_from_plt_file() :
    activity = 0
    activity_id_list = []
    user_id_list = []
    lat_list = []
    long_list = []
    altitude_list = []
    date_days_list = []
    current_date_time_list = []
    start_date_time_list = []
    end_date_time_list = []

    for dirpath, dirnames, filenames in os.walk(os.getcwd()):
        for filename in filenames:

            # we get the information of each plt file
            if filename.endswith('.plt'):
                try :
                    with open(os.path.join(dirpath, filename)) as f:
                        lines = f.readlines()

                        # skip the first 6 lines
                        lines = lines[6:]

                        # check the length of the plt file
                        if len(lines) <= 2500:

                            # we get the start and end date of each plt file
                            start_line= lines[0].split(',')
                            start_date = start_line[5]
                            start_time = start_line[6]
                            start_datetime = start_date + ' ' + start_time
                            start_datetime = start_datetime.rstrip('\n')
                            start_datetime = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")

                            end_line = lines[len(lines)-1].split(',')
                            end_date = end_line[5]
                            end_time = end_line[6]
                            end_datetime = end_date + ' ' + end_time
                            end_datetime = end_datetime.rstrip('\n')
                            end_datetime = datetime.strptime(end_datetime, "%Y-%m-%d %H:%M:%S")
    
                            # we save the information of each line of the plt file
                            for line in lines:

                                data = line.split(',')

                                activity_id_list.append(activity)

                                parent_directory = os.path.dirname(dirpath)
                                directory_name = os.path.basename(parent_directory) 
                                user_id_list.append(directory_name)

                                lat_list.append(float(data[0]))
                                long_list.append(float(data[1]))
                                altitude_list.append(int(float(data[3])))
                                date_days_list.append(float(data[4]))

                                date = data[5]
                                time = data[6]
                                datetime_draft = date + ' ' + time
                                datetime_draft = datetime_draft.rstrip('\n')
                                combined_datetime = datetime.strptime(datetime_draft, "%Y-%m-%d %H:%M:%S")
                                current_date_time_list.append(combined_datetime)

                                start_date_time_list.append(start_datetime)
                                end_date_time_list.append(end_datetime)

                    f.close()

                    activity +=1
                    
                except FileNotFoundError:
                    print(f"the file {filename} doesn't exist.")
                except Exception as e:
                    print(f"Error: {e}")
                    
    return activity_id_list, user_id_list, lat_list, long_list, altitude_list, date_days_list, current_date_time_list, start_date_time_list, end_date_time_list

In [83]:
activity_id_list, user_id_list_2, lat_list, long_list, altitude_list, date_days_list, current_date_time_list, start_date_time_list2, end_date_time_list2 = get_info_from_plt_file()

# checking if the lists have the same length
print('len activity: ' + str(len(activity_id_list)))
print('len user: ' + str(len(user_id_list_2)))
print('len lat: ' + str(len(lat_list)))
print('len long: ' + str(len(long_list)))
print('len altitude: ' + str(len(altitude_list)))
print('len date_days: ' + str(len(date_days_list)))
print('len date_time: ' + str(len(current_date_time_list)))
print('len start date_time: ' + str(len(start_date_time_list2)))
print('len end date_time: ' + str(len(end_date_time_list2)))

# converting into a dataframe
plt = {'activity_id': activity_id_list, 'user_id': user_id_list_2, 'lat': lat_list, 'long': long_list, 'altitude': altitude_list, 'date_days': date_days_list, 'current_date_time': current_date_time_list, 'start_date_time': start_date_time_list2, 'end_date_time': end_date_time_list2}
plt_df = pd.DataFrame(plt)

len activity: 9681756
len user: 9681756
len lat: 9681756
len long: 9681756
len altitude: 9681756
len date_days: 9681756
len date_time: 9681756
len start date_time: 9681756
len end date_time: 9681756


In [84]:
plt_df

,activity_id,user_id,lat,long,altitude,date_days,current_date_time,start_date_time,end_date_time
0,0,135,39.974294,116.399741,492,39816.056644,2009-01-03 01:21:34,2009-01-03 01:21:34,2009-01-03 05:40:31
1,0,135,39.974292,116.399592,492,39816.056655,2009-01-03 01:21:35,2009-01-03 01:21:34,2009-01-03 05:40:31
2,0,135,39.974309,116.399523,492,39816.056667,2009-01-03 01:21:36,2009-01-03 01:21:34,2009-01-03 05:40:31
3,0,135,39.974320,116.399588,492,39816.056690,2009-01-03 01:21:38,2009-01-03 01:21:34,2009-01-03 05:40:31
4,0,135,39.974365,116.399730,491,39816.056701,2009-01-03 01:21:39,2009-01-03 01:21:34,2009-01-03 05:40:31
...,...,...,...,...,...,...,...,...,...
9681751,18668,171,39.973817,116.324117,265,39286.418252,2007-07-23 10:02:17,2007-07-23 03:31:53,2007-07-23 10:06:38
9681752,18668,171,39.973333,116.323683,265,39286.419028,2007-07-23 10:03:24,2007-07-23 03:31:53,2007-07-23 10:06:38
9681753,18668,171,39.973733,116.323250,364,39286.419421,2007-07-23 10:03:58,2007-07-23 03:31:53,2007-07-23 10:06:38
9681754,18668,171,39.973317,116.322967,246,39286.420243,2007-07-23 10:05:09,2007-07-23 03:31:53,2007-07-23 10:06:38


In [25]:
class Task1:

    # we use constant for defining the tables
    USER = 'User'
    ACTIVITY = 'Activity'
    TRACK_POINT = 'TrackPoint'

    def __init__(self):
        self.connection = DbConnector()
        self.db_connection = self.connection.db_connection
        self.cursor = self.connection.cursor

# --------------- CREATION OF THE TABLES ---------------

    def create_table_user(self):
        query = """CREATE TABLE IF NOT EXISTS %s (
                    userID VARCHAR(30) NOT NULL PRIMARY KEY,
                    has_labels BOOLEAN)
                """
        # This adds table_name to the %s variable and executes the query
        self.cursor.execute(query % self.USER)
        self.db_connection.commit()
    

    def create_table_activity(self):
        query = """CREATE TABLE IF NOT EXISTS %s (
                    activityID int NOT NULL,
                    userID varchar(30) NOT NULL,	
                    transportation_mode varchar(30),
                    start_date_time datetime NOT NULL,
                    end_date_time datetime NOT NULL,
                    PRIMARY KEY (activityID),
                    FOREIGN KEY (userID) REFERENCES User(userID)
                )"""
        # This adds table_name to the %s variable and executes the query
        self.cursor.execute(query % self.ACTIVITY)
        self.db_connection.commit()

        query_constr = """ ALTER TABLE %s
                    ADD CONSTRAINT check_start_end_dates
                    CHECK (start_date_time < end_date_time)
                """

        self.cursor.execute(query_constr % self.ACTIVITY)
        self.db_connection.commit()
    

    def create_table_trackpoint(self):
        query = """CREATE TABLE IF NOT EXISTS %s (
                    trackPointID INT AUTO_INCREMENT NOT NULL,
                    activityID int NOT NULL,
	                lat double NOT NULL,
	                lon double NOT NULL,
	                altitude int NOT NULL,
	                date_days double NOT NULL,
	                date_time datetime NOT NULL,
	                PRIMARY KEY (trackPointID),
	                FOREIGN KEY (activityID) REFERENCES Activity(activityID))
                """
        # This adds table_name to the %s variable and executes the query
        self.cursor.execute(query % self.TRACK_POINT)
        self.db_connection.commit()

# --------------- INSERT OF DATA ---------------

# *** USER ***
    def insert_data_user(self):
        print('*** INSERT IN USER ***')
        # we go to the Data directory
        path = os.getcwd()
        os.chdir(path+"\\dataset\\dataset\\Data") #FIXME: maybe you need to change if
        
        # we get the list of all the diferent directory names (users ids) and we sort the list
        usersID =  os.listdir()
        usersID.sort()

        # we go back to the dataset directory and we read the labeled_ids.txt file
        os.chdir("..")
        with open('labeled_ids.txt') as f:
            labeled_ids = f.readlines()
        f.close()
        os.chdir("..")
        os.chdir("..")
        
        # # we delete the \n in each string
        for i in range (0, len(labeled_ids)):
            labeled_ids[i] = labeled_ids[i].strip()

        # we check if each user has a label or not and we save the info in a list
        # the indexes of has_labels and id lists are correponding
        has_labels = []
        for i in usersID : 
            if i in labeled_ids : 
                has_labels.append(True)
            else:
                has_labels.append(False)

        # we execute the query to insert the data
        for i in range(len(usersID)):

            # NOTE: if we want to insert a string -> '%s'
            query = "INSERT INTO %s (userID, has_labels) VALUES ('%s', %s)"
            values = (self.USER, usersID[i], has_labels[i])
            self.cursor.execute(query % (values))

        self.db_connection.commit()
    
# *** ACTIVITY ***
    def inser_data_activity(self):
        print('*** INSERT IN ACTIVITY ***')
        user_id_list
        transportation_mode_list
        start_date_time_list

        #TODO 
        # [x] devo ciclare su tutti i plt file
        # [x] controllare che non siano più lunghi di 2500 righe
        # [ ] controllare se quel plt file ha una corrispondenza in label
        #       [ ] se ha una corrispondenza vedere se la prima e l'ultima riga del plt file
        #           corrispondono con start e end time
        #           [ ] se il check è ok -> aggiungere a transportation mode la label corretta
        #       [ ] se non ha una corrispondenza aggiungere il track point con 
        #           transportation mode = NULL            


        self.db_connection.commit()


# --------------- FETCH DATA ---------------

    def fetch_data(self, table_name):
        query = "SELECT * FROM %s"
        self.cursor.execute(query % table_name)
        rows = self.cursor.fetchall()
        print("Data from table %s, raw format:" % table_name)
        print(rows)
        # Using tabulate to show the table in a nice way
        print("Data from table %s, tabulated:" % table_name)
        print(tabulate(rows, headers=self.cursor.column_names))
        return rows

# --------------- DROP TABLE ---------------

    def drop_table(self, table_name):
        print("Dropping table %s..." % table_name)
        query = "DROP TABLE %s"
        self.cursor.execute(query % table_name)

# --------------- SHOW TABLES ---------------

    def show_tables(self):
        self.cursor.execute("SHOW TABLES")
        rows = self.cursor.fetchall()
        print(tabulate(rows, headers=self.cursor.column_names))


In [26]:
def main():
    program = None

    try:
        program = Task1() 

        # --- DROP TABLE ---
        program.drop_table(program.TRACK_POINT)
        program.drop_table(program.ACTIVITY)
        program.drop_table(program.USER)

        # --- TABLE CREATION ---
        # NOTE: already created
        program.create_table_user()
        program.create_table_activity()
        program.create_table_trackpoint()

        # --- INSERTION ---
        # NOTE: already inserted
        program.insert_data_user()
        program.inser_data_activity()

        

        # --- FETCH ---
        #program.fetch_data(program.USER)
        
        # --- SHOW TABLES ---
        #program.show_tables()
        
    except Exception as e:
        print("ERROR: Failed to use database:", e)
    finally:
        if program:
            program.connection.close_connection()

if __name__ == '__main__':
    main()

Connected to: 8.0.34-0ubuntu0.22.04.1
You are connected to the database: ('test_19',)
-----------------------------------------------

*** INSERT IN ACTIVITY ***
Data from table User, raw format:
[('000', 0), ('001', 0), ('002', 0), ('003', 0), ('004', 0), ('005', 0), ('006', 0), ('007', 0), ('008', 0), ('009', 0), ('010', 1), ('011', 0), ('012', 0), ('013', 0), ('014', 0), ('015', 0), ('016', 0), ('017', 0), ('018', 0), ('019', 0), ('020', 1), ('021', 1), ('022', 0), ('023', 0), ('024', 0), ('025', 0), ('026', 0), ('027', 0), ('028', 0), ('029', 0), ('030', 0), ('031', 0), ('032', 0), ('033', 0), ('034', 0), ('035', 0), ('036', 0), ('037', 0), ('038', 0), ('039', 0), ('040', 0), ('041', 0), ('042', 0), ('043', 0), ('044', 0), ('045', 0), ('046', 0), ('047', 0), ('048', 0), ('049', 0), ('050', 0), ('051', 0), ('052', 1), ('053', 1), ('054', 0), ('055', 0), ('056', 1), ('057', 0), ('058', 1), ('059', 1), ('060', 1), ('061', 0), ('062', 1), ('063', 0), ('064', 1), ('065', 1), ('066', 0),